ZnTrack allos us to use other Nodes as parameters. The following Example will show, how a Node can be used in the graph or as a `zn.nodes` depedency.

In [1]:
from zntrack import config

config.nb_name = "nodes_for_parameters.ipynb"

In [2]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [3]:
!git init
!dvc init

Initialized empty Git repository in /tmp/tmpm_wtcyt9/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [4]:
import zntrack
import json

In [5]:
class SliceData(zntrack.Node):
    start: int = zntrack.zn.params()
    stop: int = zntrack.zn.params()
    step: int = zntrack.zn.params()

    data_file = zntrack.dvc.deps()
    outs = zntrack.zn.outs()

    def run(self):
        with open(self.data_file, "r") as f:
            data = json.load(f)
        self.outs = self.slice_data(data)

    def slice_data(self, data):
        return data[self.start : self.stop : self.step]

In [6]:
with open("data.json", "w") as f:
    json.dump(list(range(100)), f)

In [7]:
with zntrack.Project() as project:
    data = SliceData(start=10, stop=20, step=2, data_file="data.json")
project.run()

Running DVC command: 'stage add --name SliceData --force ...'
Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
[NbConvertApp] Converting notebook nodes_for_parameters.ipynb to script
[NbConvertApp] Writing 1494 bytes to nodes_for_parameters.py
Running DVC command: 'repro'


In [8]:
data.load()
data.outs

[10, 12, 14, 16, 18]

In [9]:
class GenerateData(zntrack.Node):
    size: int = zntrack.zn.params()
    slicing: SliceData = zntrack.zn.nodes()

    outs = zntrack.zn.outs()

    def run(self):
        data = list(range(self.size))
        self.outs = self.slicing.slice_data(data)

In [10]:
slicer = SliceData(start=10, stop=20, step=2, data_file=None)
with zntrack.Project() as project:
    data = GenerateData(size=100, slicing=slicer)
project.run()

Running DVC command: 'stage add --name GenerateData --force ...'
Running DVC command: 'stage add --name GenerateData_slicing --force ...'
Could not create .gitignore entry in /tmp/tmpm_wtcyt9/nodes/GenerateData_slicing/.gitignore. DVC will attempt to create .gitignore entry again when the stage is run.
[NbConvertApp] Converting notebook nodes_for_parameters.ipynb to script
[NbConvertApp] Writing 1494 bytes to nodes_for_parameters.py
[NbConvertApp] Converting notebook nodes_for_parameters.ipynb to script
[NbConvertApp] Writing 1494 bytes to nodes_for_parameters.py
Running DVC command: 'repro'


In [11]:
data.load()
data.outs

[10, 12, 14, 16, 18]

In [12]:
data.slicing

SliceData(data_file=None, name='GenerateData_slicing', start=10, step=2, stop=20)

We now have two seperated subgraphs. Note, that the `GenerateData_slicing` will only serve as a parameters and method Node. The `run` of `GenerateData_slicing` won't be executed.

[![](https://mermaid.ink/img/pako:eNpdjz8PgkAMxb8K6SyDut3ghHFywklrTMMVuOT-kKPEGMJ39wIhGra-33t9TUeogmZQUNvwrlqKkt0K9Cg-4f0DobSm4oKEEJ4rPyR-Yc-RZGsdN9arT_vGN_-RPD_NLbADx9GR0en-iD7LEKRlxwgqjZprGqwgoJ9SdOh06jxrIyGCkjjwDmiQUH58teolUxhqIjlQNdk-0Y78PYSf5rnjuvw9vz99Aco_XDs?type=png)](https://mermaid.live/edit#pako:eNpdjz8PgkAMxb8K6SyDut3ghHFywklrTMMVuOT-kKPEGMJ39wIhGra-33t9TUeogmZQUNvwrlqKkt0K9Cg-4f0DobSm4oKEEJ4rPyR-Yc-RZGsdN9arT_vGN_-RPD_NLbADx9GR0en-iD7LEKRlxwgqjZprGqwgoJ9SdOh06jxrIyGCkjjwDmiQUH58teolUxhqIjlQNdk-0Y78PYSf5rnjuvw9vz99Aco_XDs)

In [10]:
temp_dir.cleanup()